# Imports

In [71]:
import random
import copy
import math
import matplotlib.pyplot as plt


# Graph dicts

In [72]:
# A dictionary mapping graph filenames to their known chromatic numbers
GRAPH_DICT = {
    'myciel3.col': 4,
    'queen8_8.col': 9,
    'le450_15b.col': 15
}
#PRINT_EACH = 50  # Number of generations between print statements
CHROMATIC_MARGIN = 5 # Margin above known chromatic number to start with

# Declaring Functions

## Parse graph from .col file (Dataset)

In [73]:
def parse_col_file(filepath):
    adj = {}
    num_vertices = 0

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('p'):
                _, _, n, _ = line.split()
                num_vertices = int(n)
                adj = {i: set() for i in range(1, num_vertices + 1)}

            elif line.startswith('e'):
                _, u, v = line.split()
                u, v = int(u), int(v)
                adj[u].add(v)
                adj[v].add(u)
    return adj, num_vertices


## Fitness and Conflict Evaluation

In [74]:
def count_conflicts(adj, coloring):
    conflicts = 0
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts += 1
    return conflicts

In [75]:
def num_colors_used(coloring):
    return len(set(coloring.values()))

In [ ]:
def fitness(adj, coloring, alpha=1, beta=10):
    return alpha * count_conflicts(adj, coloring) + beta * num_colors_used(coloring)

## Population Init

In [77]:
def random_coloring(num_vertices, max_colors):
    return {i: random.randint(1, max_colors) for i in range(1, num_vertices + 1)}

In [78]:
def initialize_population(pop_size, num_vertices, max_colors):
    return [random_coloring(num_vertices, max_colors) for _ in range(pop_size)]

# Selection

## Tournament Selection

In [79]:
def tournament_selection(population, adj, k=3):
    candidates = random.sample(population, k)
    return min(candidates, key=lambda c: fitness(adj, c))

## Roulette wheel

In [ ]:
def roulette_wheel_selection(population, adj, epsilon=1e-6):
    # Compute fitness values
    fitness_values = [fitness(adj, c) for c in population]

    # Convert to selection scores (invert fitness)
    scores = [1.0 / (f + epsilon) for f in fitness_values]

    total_score = sum(scores)
    pick = random.uniform(0, total_score)

    current = 0.0
    for individual, score in zip(population, scores):
        current += score
        if current >= pick:
            return individual

    # Fallback (numerical safety)
    return population[-1]


# Crossover

## Uniform Crossover

In [80]:
def uniform_crossover(parent1, parent2):
    child = {}
    for v in parent1:
        child[v] = parent1[v] if random.random() < 0.5 else parent2[v]
    return child


## Two-Point Crossover

In [ ]:
def two_point_crossover(parent1, parent2):
    vertices = list(parent1.keys())
    c1, c2 = sorted(random.sample(range(len(vertices)), 2))

    child = {}
    for i, v in enumerate(vertices):
        if c1 <= i < c2:
            child[v] = parent2[v]
        else:
            child[v] = parent1[v]

    return child


# Mutation

## Probabilistic, Conflict-Driven mutation

In [ ]:
def conflict_based_mutation(coloring, adj, max_colors, mutation_rate=0.05):
    for u in adj:
        conflicts = [v for v in adj[u] if coloring[u] == coloring[v]]
        if conflicts and random.random() < mutation_rate:
            coloring[u] = random.randint(1, max_colors)
    return coloring


## Single gene mutation

In [ ]:
def single_gene_mutation(coloring, max_colors):
    v = random.choice(list(coloring.keys()))
    coloring[v] = random.randint(1, max_colors)
    return coloring


# Genetic Algorithm

## Definition of the Genetic Function

In [82]:
def genetic_algorithm(
    adj,
    num_vertices,
    max_colors,
    pop_size=100,
    generations=1000,
    elitism_rate=0.05,
    mutation_rate=0.02
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number 
    if chromatic_number < 5:
        print_each = 1
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 10
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 50
    population = initialize_population(pop_size, num_vertices, max_colors) # Initial population
    elite_size = int(pop_size * elitism_rate) # Number of elites
    # initialize best solution tracking
    best_solution = None
    best_fitness = float('inf')
    found_feasible = False
    conflicts = []
    
    # main GA loop
    for gen in range(generations):
        population.sort(key=lambda c: fitness(adj, c))  # Sort by fitness
        elites = population[:elite_size]               # Select elites

        current_best = population[0]
        current_fitness = fitness(adj, current_best)
        current_conflicts = count_conflicts(adj, current_best)
        current_colors = num_colors_used(current_best)

        # Update global best
        if current_fitness < best_fitness:                  # New best found
            best_solution = copy.deepcopy(current_best)     # Update best solution
            best_fitness = current_fitness                  # Update best fitness

        if gen % print_each == 0:   # Print every set number of generations
            conflicts.append(current_conflicts)
            print(
                f"Gen {gen} | Best fitness: {best_fitness} | "
                f"Conflicts: {count_conflicts(adj, best_solution)} | "
                f"Colors: {num_colors_used(best_solution)}"
            )

        # First feasible solution (any number of colors)
        if current_conflicts == 0 and not found_feasible:
            print(f"Feasible solution found at generation {gen}")
            found_feasible = True

        # Early stopping condition
        if current_conflicts == 0 and current_colors <= chromatic_number:   # Optimal solution found, colors equal chromatic number and no conflicts
            print(
                f"Optimal solution found at generation {gen} | "
                f"Colors: {current_colors}"
            )
            return current_best, conflicts

        new_population = elites.copy()  # Start new population with elites (unchanged)

        while len(new_population) < pop_size:   # Fill the rest of the population using tournament selection, crossover, and mutation
            p1 = tournament_selection(population, adj)
            p2 = tournament_selection(population, adj)

            child = uniform_crossover(p1, p2)
            mutate(child, adj, max_colors, mutation_rate)

            new_population.append(child)

        population = new_population

    return best_solution, conflicts


## Usage of the Genetic Algorithm

In [83]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 55)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 55)

        solution, conflicts = genetic_algorithm(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            pop_size=200,
        )

        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)



myciel3.col: n=11, chromatic_number=4 max_colors=9
Gen 0 | Best fitness: 6 | Conflicts: 0 | Colors: 6
Feasible solution found at generation 0
Gen 1 | Best fitness: 5 | Conflicts: 0 | Colors: 5
Gen 2 | Best fitness: 4 | Conflicts: 0 | Colors: 4
Optimal solution found at generation 2 | Colors: 4
Final solution:
Conflicts: 0
Colors used: 4
queen8_8.col: n=64, chromatic_number=9 max_colors=14
Gen 0 | Best fitness: 50 | Conflicts: 36 | Colors: 14
Gen 10 | Best fitness: 20 | Conflicts: 6 | Colors: 14
Gen 20 | Best fitness: 19 | Conflicts: 5 | Colors: 14
Gen 30 | Best fitness: 19 | Conflicts: 5 | Colors: 14
Gen 40 | Best fitness: 19 | Conflicts: 5 | Colors: 14
Gen 50 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 60 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Feasible solution found at generation 68
Gen 70 | Best fitness: 14 | Conflicts: 0 | Colors: 14
Gen 80 | Best fitness: 14 | Conflicts: 0 | Colors: 14
Gen 90 | Best fitness: 14 | Conflicts: 0 | Colors: 14
Gen 100 | Best fitness: 14 

# Tabu Search

## Implementation of an optimized conflict storage
Using a data structure reduces the amount of computations since the tabu search requires not only the amount of conflicts but where have they appeared in order to generate meaningful neighborhood moves, because only recoloring conflicted vertices can reduce conflicts.
The total conflict count alone provides no structural information about where the problem is, making targeted local search impossible.

In [84]:
def conflicting_vertices(adj, coloring):
    conflicts = set()
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts.add(u)
                conflicts.add(v)
    return list(conflicts)


## Implementation of the algorithm

In [ ]:
def tabu_search(
    adj,
    num_vertices,
    max_colors,
    max_iterations=20000,
    tabu_tenure=7,
    aspiration=True,
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number
    if chromatic_number < 5:
        print_each = 50
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 100
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 500
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors
    found_optimal = False
    tabu_list = {}
    iteration = 0

    while iteration < max_iterations and not found_optimal:

        conflicted = conflicting_vertices(adj, current)
        best_move = None
        best_score = (float("inf"), float("inf"))

        for v in conflicted:
            original_color = current[v]

            for c in range(1, max_colors + 1):
                if c != original_color:

                    current[v] = c
                    new_conflicts = count_conflicts(adj, current)
                    new_colors = num_colors_used(current)

                    score = (new_conflicts, new_colors)
                    delta_better = score < best_score

                    move = (v, c)
                    tabu = move in tabu_list and tabu_list[move] > iteration

                    if tabu and aspiration:
                        if (new_conflicts < best_conflicts or
                           (new_conflicts == best_conflicts and new_colors < best_colors)):
                            tabu = False

                    if not tabu and delta_better:
                        best_score = score
                        best_move = (v, c, new_conflicts, new_colors)

                    current[v] = original_color

        if best_move is not None:
            v, c, new_conflicts, new_colors = best_move
            old_color = current[v]

            current[v] = c
            current_conflicts = new_conflicts
            current_colors = new_colors

            tabu_list[(v, old_color)] = iteration + tabu_tenure

            # Update global best using lexicographic objective
            if (current_conflicts < best_conflicts or
               (current_conflicts == best_conflicts and current_colors < best_colors)):
                best = copy.deepcopy(current)
                best_conflicts = current_conflicts
                best_colors = current_colors

        if iteration % print_each == 0:
            print(
                f"Iter {iteration} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )
        # Early stopping condition
        if best_conflicts == 0 and best_colors <= chromatic_number:
            print(
                f"Optimal solution found at iteration {iteration} | "
                f"Colors: {best_colors}"
            )
            found_optimal = True

        iteration += 1

    return best


## Usage of Tabu Search

In [86]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 50)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 50)

        solution = tabu_search(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            max_iterations=2000,
            tabu_tenure=7,
        )
        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)


myciel3.col: n=11, chromatic_number=4 max_colors=9
Iter 0 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 1 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 2 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 3 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 4 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 5 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 6 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 7 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 8 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 9 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 10 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 11 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 12 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 13 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 14 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 15 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 16 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 17 | Conflicts: 0 | Colors: 8 | Best colors: 8
Iter 18 | Conflicts: 0 

# Simmulated Annealing

## Algorithm implementation

In [87]:
def simulated_annealing(
    adj,
    num_vertices,
    max_colors,
    vert_list,
    max_iterations=50000,
    initial_temp=10.0,
    cooling_rate=0.9995,
    conflict_weight=1000,
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number
    if chromatic_number < 5:
        print_each = 100
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 500
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 1000
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)
    found_optimal = False
    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    temperature = initial_temp
    iteration = 0

    while (iteration < max_iterations and temperature > 1e-6) and not found_optimal:

        # Pick a vertex biased toward conflicts
        conflicted = conflicting_vertices(adj, current)
        if len(conflicted) > 0:
            v = random.choice(conflicted)
        else:
            v = random.choice(vert_list)

        original_color = current[v]
        new_color = random.randint(1, max_colors)

        if new_color != original_color:

            current[v] = new_color
            new_conflicts = count_conflicts(adj, current)
            new_colors = num_colors_used(current)

            delta = (
                (new_conflicts - current_conflicts) * conflict_weight
                + (new_colors - current_colors)
            )

            accept = False
            if delta <= 0:
                accept = True
            else:
                prob = math.exp(-delta / temperature)
                accept = random.random() < prob

            if accept:
                current_conflicts = new_conflicts
                current_colors = new_colors

                if (
                    current_conflicts < best_conflicts or
                    (current_conflicts == best_conflicts and current_colors < best_colors)
                ):
                    best = copy.deepcopy(current)
                    best_conflicts = current_conflicts
                    best_colors = current_colors
            else:
                current[v] = original_color

        temperature *= cooling_rate
        iteration += 1

        if iteration % print_each == 0:
            print(
                f"Iter {iteration} | "
                f"T={temperature:.4f} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )
        # Early stopping condition
        if best_conflicts == 0 and best_colors <= chromatic_number:
            print(
                f"Optimal solution found at iteration {iteration} | "
                f"Colors: {best_colors}"
            )
            found_optimal = True

    return best


## Usage of Simulated Annealing

In [88]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)
        vert_list = list(adj.keys())

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 50)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 50)

        solution = simulated_annealing(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            vert_list=vert_list
        )

        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)   


myciel3.col: n=11, chromatic_number=4 max_colors=9
Iter 100 | T=9.5122 | Conflicts: 0 | Colors: 6 | Best colors: 5
Iter 200 | T=9.0481 | Conflicts: 0 | Colors: 9 | Best colors: 5
Iter 300 | T=8.6068 | Conflicts: 0 | Colors: 7 | Best colors: 5
Iter 400 | T=8.1869 | Conflicts: 0 | Colors: 8 | Best colors: 5
Iter 500 | T=7.7875 | Conflicts: 0 | Colors: 6 | Best colors: 5
Iter 600 | T=7.4076 | Conflicts: 0 | Colors: 7 | Best colors: 5
Iter 700 | T=7.0463 | Conflicts: 0 | Colors: 8 | Best colors: 5
Iter 800 | T=6.7025 | Conflicts: 0 | Colors: 6 | Best colors: 5
Iter 900 | T=6.3756 | Conflicts: 0 | Colors: 7 | Best colors: 5
Iter 1000 | T=6.0645 | Conflicts: 0 | Colors: 7 | Best colors: 5
Iter 1100 | T=5.7687 | Conflicts: 0 | Colors: 6 | Best colors: 5
Iter 1200 | T=5.4873 | Conflicts: 0 | Colors: 6 | Best colors: 5
Iter 1300 | T=5.2196 | Conflicts: 0 | Colors: 8 | Best colors: 5
Iter 1400 | T=4.9650 | Conflicts: 0 | Colors: 7 | Best colors: 5
Iter 1500 | T=4.7228 | Conflicts: 0 | Colors: 7 